In [1]:
# import libraries
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

import numpy as np
import os
import time
import random

# set gpu devices if available
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices:
    print(device)
    tf.config.experimental.set_memory_growth(device, True)

2021-11-25 19:32:45.286065: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


2021-11-25 19:32:46.010964: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-11-25 19:32:46.011488: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-11-25 19:32:46.077599: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-25 19:32:46.077826: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:09:00.0 name: NVIDIA GeForce RTX 3060 Ti computeCapability: 8.6
coreClock: 1.695GHz coreCount: 38 deviceMemorySize: 7.77GiB deviceMemoryBandwidth: 417.29GiB/s
2021-11-25 19:32:46.077846: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-11-25 19:32:46.080720: I tensorflow/stream_executor/platform

In [2]:
text = open('dataset.txt', 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')
# the unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

Length of text: 858397 characters
117 unique characters


In [3]:
# methods to converte between ids and chars
ids_from_chars = preprocessing.StringLookup(
    vocabulary=list(vocab), mask_token=None)
chars_from_ids = tf.keras.layers.experimental.preprocessing.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)
# create ids
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

2021-11-25 19:32:47.873189: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-11-25 19:32:47.873351: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-25 19:32:47.873661: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:09:00.0 name: NVIDIA GeForce RTX 3060 Ti computeCapability: 8.6
coreClock: 1.695GHz coreCount: 38 deviceMemorySize: 7.77GiB deviceMemoryBandwidth: 417.29GiB/s
2021-11-25 19:32:47.873685: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-11-25 19:32:47.873701: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2021-11-25 19:32:47.873707: I tensorflow/stream_executor/platf

In [4]:
# define sequence length
seq_length = 100
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)
# create proper dataset
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text
dataset = sequences.map(split_input_target)

In [5]:
# batch size
BATCH_SIZE = 280

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset shapes: ((280, 100), (280, 100)), types: (tf.int64, tf.int64)>

In [6]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

# create model
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [7]:
# customize training
class CustomTraining(MyModel):
  @tf.function
  def train_step(self, inputs):
      inputs, labels = inputs
      with tf.GradientTape() as tape:
          predictions = self(inputs, training=True)
          loss = self.loss(labels, predictions)
      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

      return {'loss': loss}

In [8]:
# instantiate and train the model
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))
model.fit(dataset, epochs=150)  # start with a small number of epochs to check performance

Epoch 1/150


2021-11-25 19:32:56.086597: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-11-25 19:32:56.117286: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3792695000 Hz
2021-11-25 19:32:56.223347: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2021-11-25 19:32:56.663646: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11
2021-11-25 19:32:56.667539: I tensorflow/stream_executor/cuda/cuda_blas.cc:1838] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


30/30 [==============================] - 6s 145ms/step - loss: 3.5998
Epoch 2/150
30/30 [==============================] - 5s 136ms/step - loss: 2.7299
Epoch 3/150
30/30 [==============================] - 5s 137ms/step - loss: 2.3675
Epoch 4/150
30/30 [==============================] - 5s 137ms/step - loss: 2.2333
Epoch 5/150
30/30 [==============================] - 5s 138ms/step - loss: 2.1459
Epoch 6/150
30/30 [==============================] - 5s 145ms/step - loss: 2.0575
Epoch 7/150
30/30 [==============================] - 5s 158ms/step - loss: 1.9703
Epoch 8/150
30/30 [==============================] - 5s 152ms/step - loss: 1.8934
Epoch 9/150
30/30 [==============================] - 5s 158ms/step - loss: 1.8214
Epoch 10/150
30/30 [==============================] - 5s 157ms/step - loss: 1.7524
Epoch 11/150
30/30 [==============================] - 5s 151ms/step - loss: 1.6837
Epoch 12/150
30/30 [==============================] - 5s 150ms/step - loss: 1.6177
Epoch 13/150
30/30 [=====

Epoch 100/150
30/30 [==============================] - 5s 143ms/step - loss: 0.2750
Epoch 101/150
30/30 [==============================] - 5s 143ms/step - loss: 0.2193
Epoch 102/150
30/30 [==============================] - 5s 143ms/step - loss: 0.1743
Epoch 103/150
30/30 [==============================] - 5s 143ms/step - loss: 0.1424
Epoch 104/150
30/30 [==============================] - 5s 143ms/step - loss: 0.1176
Epoch 105/150
30/30 [==============================] - 5s 143ms/step - loss: 0.1015
Epoch 106/150
30/30 [==============================] - 5s 151ms/step - loss: 0.0920
Epoch 107/150
30/30 [==============================] - 5s 143ms/step - loss: 0.0867
Epoch 108/150
30/30 [==============================] - 5s 143ms/step - loss: 0.0834
Epoch 109/150
30/30 [==============================] - 5s 143ms/step - loss: 0.0817
Epoch 110/150
30/30 [==============================] - 5s 143ms/step - loss: 0.0806
Epoch 111/150
30/30 [==============================] - 5s 143ms/step - loss:

In [9]:
# Step class
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [10]:
# finally use the model to generate text
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)
start = time.time()
states = None
next_char = tf.constant(['La felicidad es'])
result = [next_char]

for n in range(3000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

La felicidad es una cosa que no siempre la guiara. Y uno no puede pasarse toda mujer como si no imposible propósito.
- Zygmunt Bauman
Dramen dolor, tampoco negaria deabulde a causa de los inversorables. Ya no mira con lo más breve: el presente, sin hobribirse o admirable, pues muchas cosas, pero has fallado en lo esencial, es decir, has fracasado. Espero está presentado en nuestro amor de Cristos y ses injurias, sin pensamientos, la justicia. Comenza el presente, para sí mismo es uno de los que tienen miedo a la oscuridad
- Stephen Hawking
He estado inmerso es el derecho que nada puede hacer durante ocho horas, día tras día a quienes nos amamos. El amor es demilisar.
- Martin Luther King
Quizá sueña se alimenta de supersoda en vuestro corazón; una buena actividad cree.
- Eurípides
Pero la vida no es la que uno vivió, sino la que uno recuerda, y cómo la relativida como cuanto más repedente que la pérdida del paraíco del asceso de su vida es despertar la verdad...
- Mario Benedetti
En el